# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1.

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [39]:
# импортируем необходимые библиотеки
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
# response 200 говорит о том, что все работает корректно 
result = requests.get('https://habr.com/ru/all/')
result

<Response [200]>

In [3]:
# ключевые слова для поиска и ссылка
KEYWORDS = ['python', 'парсинг']
link_ = 'https://habr.com/ru/all/'

In [4]:
# получаем страницу с самыми свежими постами
result = requests.get(link_)
soup = BeautifulSoup(result.text, 'html.parser')

In [5]:
# получаем все статьи
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Artem_Gorshkov/" title="Автор публикации">
 <span class="default-image_mini default-image_pink">
 <svg aria-hidden="true" class="icon-svg" height="24" role="img" version="1.1" viewbox="0 0 24 24" width="24"><path d="M21.5 24h-19c-1.379 0-2.5-1.122-2.5-2.5v-19c0-1.379 1.122-2.5 2.5-2.5h19c1.379 0 2.5 1.122 2.5 2.5v19c0 1.379-1.122 2.5-2.5 2.5zm-19-23c-.827 0-1.5.673-1.5 1.5v19c0 .827.673 1.5 1.5 1.5h19c.827 0 1.5-.673 1.5-1.5v-19c0-.827-.673-1.5-1.5-1.5h-19zM15.598 12.385zM19.438 15.417l-.002-.005v-.001c-.875-2.226-2.484-3.054-3.445-3.549l-.273-.143c.029-.497-.025-1.034-.167-1.599l-.128.032.123-.044c-.765-2.152-1.757-2.585-2.632-2.967l-.006-.003-.535-2.121c.357-.065.628-.375.628-.752.001-.423-.342-.765-.765-.765s-.766.342-.766.765c0 .358.248.657.581.74l-.825 1.654-.014-.003-.024-.003c-1.053-.033-1.842.369-2.5.947-.633-.322-1.515-.729-2.158-1.8

In [6]:
# получаем ссылки и заголовки статей 
post_head = list(map(lambda x: x.find('h2', class_='post__title'), posts))
post_head

[<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/530926/">Ведущий переговоры 2020</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/530922/">Как победить гроссмейстера, не умея играть в шахматы. Про злоупотребление доказательства с нулевым знанием</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/528460/">История презентаций: как появились PowerPoint и Keynote (часть 2)</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/vdsina/blog/530870/">Проверим тысячи пакетов PyPI на вредоносность</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/nanosoft/blog/527888/">Анализ целостности сигналов в PADS Professional (2/6)</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/530914/">«Удаленка» навсегда? Опыт команд Firmware

In [7]:
# добираемся до ссылок
post_link = list(map(lambda x: x.find('a').get('href'), post_head))
post_link

['https://habr.com/ru/post/530926/',
 'https://habr.com/ru/post/530922/',
 'https://habr.com/ru/post/528460/',
 'https://habr.com/ru/company/vdsina/blog/530870/',
 'https://habr.com/ru/company/nanosoft/blog/527888/',
 'https://habr.com/ru/post/530914/',
 'https://habr.com/ru/post/530912/',
 'https://habr.com/ru/company/ua-hosting/blog/530724/',
 'https://habr.com/ru/company/pixonic/blog/530906/',
 'https://habr.com/ru/company/yandex/blog/530844/',
 'https://habr.com/ru/company/searchinform/blog/530904/',
 'https://habr.com/ru/company/microsoft/blog/530442/',
 'https://habr.com/ru/post/530878/',
 'https://habr.com/ru/post/530900/',
 'https://habr.com/ru/company/intel/blog/530710/',
 'https://habr.com/ru/company/alexhost/blog/530862/',
 'https://habr.com/ru/company/huawei/blog/530892/',
 'https://habr.com/ru/post/530890/',
 'https://habr.com/ru/post/530888/',
 'https://habr.com/ru/company/alconost/blog/530806/']

In [15]:
# собираем даты, заголовки и тексты новостей и тексты всех статей 
for link in post_link:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    date = soup.find('span', class_='post__time').get('data-time_published')
    print(date)
    title = soup.find('span', class_='post__title-text').text
    print(title)
    text = soup.find('div', class_='post__text').text
    print(text)

2020-12-02T08:57Z
Ведущий переговоры 2020
2020 год надолго останется в нашей памяти. Мы буквально ходили «по лезвию», отклоняясь то в одну, то в другую сторону. Выходили из каратина и возвращались в него. Вспоминали каково это — видеть коллег и партнеров не в сети, видели их в уже привычных окошках Zoom'a. 
Мы «зависаем» на видеовзонках и тонем в море электронной переписки. 2020й повлиял на наше общение и впору дописывать в резюме — «эффективно провожу переговоры в удаленном формате». А как это — эффективно проводить переговоры и особенно не в live?

А вы знаете, что такое груминг? Не в контексте гибкого подхода к разработке или стрижки собак. Грумингом также называют процесс взаимного вычесывания у обезьян. С его помощью приматы выстраивают отношения друг с другом, создают социальные связи и проходят процесс социальной адаптации. Результатами этого процесса являются «условные единицы» взаимного уважения, которые учитываются в иерархической системе стаи. Для общества людей, так же как 

2020-12-02T08:48Z
Как победить гроссмейстера, не умея играть в шахматы. Про злоупотребление доказательства с нулевым знанием
Это Алиса. Она не умеет играть в шахматы, но изучает защиту информации. Поэтому она сможет обыграть гроссмейстера. Для этого ей понадобится:Два гроссмейстера,Две шахматные доски с фигурами,Две комнаты.И вот как она этого добьется.Проблема гроссмейстераАлиса приглашает Гарри Каспарова и Анатолия Карпова сыграть с ней в шахматы в одном месте, в одно время, но в разных комнатах. Ни Каспаров, ни Карпов, не знают о присутствии другого. Алиса играет белыми в партии с Каспаровым и черными в партии с Карповым. Сначала Карпов делает ход первым, так как он играет за белых. Алиса записывает ход, идёт в комнату к Каспарову и повторяет записанный ход. Далее записывает ответ Каспарова черными и повторяет этот ход в партии с Карповым.Так продолжается до тех пор, пока она не выиграет одну из партий, проигрывая другую. Либо обе партии заканчиваются вничью.На самом деле Каспаров и

2020-12-02T08:30Z
Проверим тысячи пакетов PyPI на вредоносность
Примерно год назад Python Software Foundation открыл Request for Information (RFI), чтобы обсудить, как можно обнаруживать загружаемые на PyPI вредоносные пакеты. Очевидно, что это реальная проблема, влияющая почти на любой менеджер пакетов: случаются захваты имён заброшенных разработчиками пакетов, эксплуатация опечаток в названиях популярных библиотек или похищение пакетов при помощи упаковки учётных данных.

Реальность такова, что менеджеры пакетов наподобие PyPI являются критически важной инфраструктурой, которой пользуется почти любая компания. Я мог бы многое написать по этой теме, но сейчас достаточно будет этого выпуска xkcd.



Эта область знаний мне интересна, поэтому я ответил своими мыслями о том, как мы можем подойти к решению проблемы. Весь пост стоит прочтения, но меня не оставляла в покое одна мысль: что происходит сразу же после установки пакета.

Такие действия, как установка сетевых соединений или исполн

2020-12-02T08:17Z
Анализ целостности сигналов в PADS Professional (2/6)

Добро пожаловать на второй урок по пред- и пост топологическому анализу в PADS Professional — Основы LineSim.

На первом уроке вы узнали как экспортировать цепи из схемы в HyperLynx LineSim и назначать модели на драйвере и приемнике.

В сегодняшнем уроке я расскажу о некоторых основных функциональных возможностях PADS Professional HyperLynx LineSim, таких как редактирование линии передачи и стека платы.

Урок 2 – Основы LineSim

В меню “ПУСК” выберите  PADS Pro Tools VX.2.x > PADS Pro Designer VX.2.x
На стартовой странице PADS Professional Designer выберите File > Open и откройте:
C:\SI_Analysis\Lesson2\HandDrillAll.prj
Повторите шаги 3-5 из первого урока для запуска PADS Professional HyperLynx LineSim
Если появится это сообщение, выберите No. После этого откроется HyperLynx LineSim.


Дважды кликните по линии передачи TL4 для ее редактирования.

убедитесь, что вы моделируете тип линии передачи Uncoupled Stackup.


2020-12-02T08:00Z
Terraform 12 и Terragrunt и как это можно применять для Multi-Cloud-инфраструктуры. Александр Довнар

Что такое Terraform 12 и Terragrunt, и как это можно применять для Multi-Cloud инфраструктуры.
Мы поговорим про IaC (Инфраструктура как код) влияние на современный мир и о том, как Terraform помогает работать с гетерогенных окружениях.Я хочу обсудить немного сам Terraform, какие у него есть проблемы и как их решает Terragrunt. После я расскажу про мой опыт с Terragrunt и немного зацеплю такую тему, как Multi-Clouds.Во второй части обсуждения темы я бы хотел показать результат моих находок в использовании Terraform+Terragrunt в среде с тремя облачными провайдерами (AWS, GCP, Azure) и CloudFlare в качестве DNS.


(Александр) Сегодня я хочу рассказать о том, как у меня получилось сделать Multi-Cloud deployment с помощью Terraform и Terragrunt, а также о том, как это работает в частности и в отдельности.

(Виктор) Круто! Я знаю, что Саша подготовил вопросы. И по традиции 

...

2020-12-02T06:44Z
Помогите стахановцу — увольте его сразу
Сидят себе люди, никого не трогают, работают потихоньку, получают деньги, выдают результат. Директор, в целом, доволен. Но чего-то не хватает – хочется больше, лучше, выше, эффективнее. Вкладываться в интенсивное развитие лень, да и знаний не хватает. Так, иногда скажет мимоходом, что надо бы лучше работать, все головой покивают, может даже составят перечень мероприятий по повышению эффективности и снижению затрат, чтобы дружно про него забыть.

Но тут приходит Он. Тот, кому Мало. Кто страстно хочет Больше. И открывает ящик Пандоры. Точнее, показывает директору, где этот ящик находится, и помогает провернуть ключ в замке.

Стахановцы
Во времена индустриализации в СССР было такое движение – стахановцы. В интернете пишут, что началось с Алексея Григорьевича Стаханова – шахтёра, который однажды сделал 14.5 норм по добыче угля. В книгах пишут, что подвиг Стаханова был создан искусственно – всех выгнали из шахты на ночь, организовали

2020-12-02T06:25Z
Intel Cryo Cooling — технология термоэлектрического охлаждения процессора


Еще одна технология Intel, наконец, достигла прилавка. На этот раз она относится к не совсем типичной для компании области — охлаждению процессоров. Нет, конечно, исторически многие ценят боксовые кулеры Intel за качество и эффективность, но экстремальных решений обычно ждут от специализированных вендоров, благо в таких на рынке нет недостатка. Без их участия тут не обошлось, и новость звучит следующим образом: начались продажи сразу нескольких систем охлаждения, созданных по технологии Intel Cryo Cooling — термоэлектрической системы на базе элемента Пельтье.

Небольшая цитата из Википедии для тех, кто, так же как и ваш покорный слуга, ранее не обращал пристального внимания на эффект Пельтье.
В основе работы элементов Пельтье лежит контакт двух полупроводниковых материалов с разными уровнями энергии электронов в зоне проводимости. При протекании тока через контакт таких материалов электрон дол

2020-12-02T05:56Z
О шахматах. И не только
Сегодня не будет тяжких раздумий о настоящем и будущем компьютерной индустрии. Сегодня я хочу рассказать об одном из своих хобби. Я играю в массу разных игр: футбол, хоккей, теннис (большой и маленький), покер, преферанс, биржа и т.п. Но мой “профильный” вид спорта — шахматы. Дальше кандидата в мастера моя карьера на этом поприще не продвинулась, но любовь к древней игре я сохраняю уже 4 десятка лет. Интересно, что она вполне “ужилась” с другим увлечением – программированием, породив интерес к искусственному интеллекту и теории игр. И разумеется, последние прорывы в этой области связанные с феноменальными успехами проекта AlphaZero не могли пройти мимо меня.


 Тогда я просто сидел и восхищался партиями AlphaZero против Stockfish. А сейчас вернулся к теме в связи с задачей оптимизации нейронных сетей, которой иногда приходится заниматься по работе (увы, меньше чем хотелось бы). Как мне кажется, задачи эти могут оказаться тесно связанными, поэто

2020-12-02T03:16Z
Самоучитель по WinCC OA. Часть 4. Повторное использование объектов. $-параметры
В предыдущей части мы завершили создание визуализиции задвижки и создали простейший скрипт, имитирующий ее поведение.У нас есть одна панель под названием Flap, которая отображает и шлет команды для одной задвижки — Flap1. Именно такая точка данных указана во всех скриптах этой панели. Возникает закономерный вопрос — что делать, если задвижек не одна? И даже не две. А несколько десятков, сотен и даже тысяч (для распределенной системы WinCC OA и несколько миллионов сигналов не помеха, смотрим на Большой Андронный Коллайдер, где применяется именно это система, и завидуем).Очевидный вариант — создать несколько десятков, сотен и тысяч панелей, где каждая точки данных приведена явно, отметаем — долго, хлопотно и грозит огромными трудозатратами в случае малейших изменений, неизбежных во время ПНР.Другой вариант, но не единственный, создание шаблона на основе имеющейся панели. Создадим копию имеющ

2020-12-02T02:50Z
Оценка трудозатрат в разработке ПО для начинающих
Оценка временных затрат при разработке ПО может быть сложной задачей, но есть несколько хитростей, которые помогут получить точные цифры.Помню, как меня впервые попросили дать оценку…Тогда это застало врасплох.Меня завели в кабинет, где были мой начальник, его босс и кто-то из вышестоящего руководства, и мы сели за круглый стол, уставившись друг на друга.Аналитики зачитали некоторые требования от клиента. Мы их коротко обсудили.И тут мой начальник повернулся ко мне и спросил: «Сколько времени это займет?»Я не знал, что сказать, — меня к этому не готовили. Мне не сказали, что на этой встрече нужно будет давать оценку — я думал, меня позвали поприсутствовать, чтобы я чему-то научился.И вдруг на меня все смотрят и ждут ответа, а я растерялся и не знаю, что сказать.Передо мной лежала бумага для заметок. Я взял один листок и начал писать какие-то числа — без понятия, для чего, но точно не для этой оценки.Где-то через минуту

In [37]:
result_df = pd.DataFrame()
for link in post_link:
    linkget = requests.get(link)
    time.sleep(0.3)
    soup = BeautifulSoup(linkget.text, 'html.parser')
    # получаем полный текст статьи
    post_full_text = soup.find('div', class_='post__body post__body_full').text
    # проверяем на наличие ключевых слов
    if any([keyword.lower() in post_full_text.lower() for keyword in KEYWORDS]):
        # находим дату поста, заголовок и полный текст
        post_date = soup.find('span', class_='post__time').get('data-time_published')
        post_title = soup.find('span', class_='post__title-text').text
        res_string = {'post_date': post_date, 'post_title': post_title, 'URL': link, 'Full_Text': post_full_text}
        result_df = pd.concat([result_df, pd.DataFrame([res_string])])
result_df

,post_date,post_title,URL,Full_Text
0,2020-12-02T08:30Z,Проверим тысячи пакетов PyPI на вредоносность,https://habr.com/ru/company/vdsina/blog/530870/,\nПримерно год назад Python Software Foundatio...
0,2020-12-02T08:00Z,Terraform 12 и Terragrunt и как это можно прим...,https://habr.com/ru/post/530912/,"\n\nЧто такое Terraform 12 и Terragrunt, и как..."
0,2020-12-02T06:59Z,Как найти что-то в тексте,https://habr.com/ru/post/530878/,\nНайти объект или распознать понятие в тексте...
0,2020-12-02T05:56Z,О шахматах. И не только,https://habr.com/ru/company/huawei/blog/530892/,\nСегодня не будет тяжких раздумий о настоящем...


## Задание 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [65]:
res = requests.post('https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data')
res

<Response [400]>

In [75]:
mail_df = pd.DataFrame()
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
data = {
    'emailAddresses': 'noob28@mail.ru',
    'Data': 'Thu, 03 Dec 2020 12:48:39 GMT'
}
headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast'
}
req = requests.post(URL, json=data, headers=headers)
mailsoup = BeautifulSoup(req.text, 'html.parser')
res_df = {'emailAddresses': data['emailAddresses'], 'Data': data['Data'], }
mail_df = pd.concat([mail_df, pd.DataFrame([res_df])])
mail_df

,emailAddresses,Data
0,noob28@mail.ru,"Thu, 03 Dec 2020 12:48:39 GMT"
